Neural Network Basics
============

Classification
---------

In [9]:
# 털과 날개의 여부에 따라 포유류인지 조류인지 분류하는 신경망 모델
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

# 신경망 모델 구성
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망은 2차원으로 [특성, 레이블] -> [2,3]으로 정합니다(가중치)
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))

# 편향은 아웃풋의 갯수, 즉 최종 결과값의 분류 갯수인 3으로 설정
b = tf.Variable(tf.zeros([3]))

# 신경망에 가중치 W와 편향 b를 적용
L = tf.add(tf.matmul(X, W), b)

# 활성화 함수 ReLU 함수 적용
L = tf.nn.relu(L)

# 마지막으로 softmax 함수를 이용하여 출력값을 사용하기 쉽게 만듭니다
# softmax 함수는 다음처럼 결과값을 전체합이 1인 확률로 만들어주는 함수입니다.
# 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
model = tf.nn.softmax(L)

# 신경망을 최적화하기 위한 비용 함수를 작성합니다.
# 각 개별 결과에 대한 합을 구한 뒤 평균을 내는 방식을 사용합니다.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis = 1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict = {X : x_data, Y : y_data})
    
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict = {X : x_data, Y : y_data}))
        
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가져옵니다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

10 1.002042
20 0.9956531
30 0.98947334
40 0.9841769
50 0.9799576
60 0.9758623
70 0.97190803
80 0.9681131
90 0.96442443
100 0.96082026
예측값: [0 2 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


Deep NN
--------

In [14]:
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array([[0,0], [1,0], [1,1], [0,0], [0,0], [0,1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫 번째 가중치 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2, 10]
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))

# 두 번째 가중치의 차원을 [첫 번째 히든 레이어의 뉴런 갯수, 분류 갯수] -> [10, 3]
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋으로 정한다. 
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

# 최종적인 아웃풋 계산
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

model = tf.add(tf.matmul(L1, W2), b2)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model)
)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict = {X : x_data, Y : y_data})
    
    if (step+1) % 10 == 0:
        print(step+1, sess.run(cost, feed_dict = {X : x_data, Y : y_data}))
        
# 결과 확인
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값 :', sess.run(prediction, feed_dict = {X : x_data, Y : y_data}))
print('실제값 :', sess.run(target, feed_dict = {X : x_data, Y : y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 :', sess.run(accuracy * 100, feed_dict = {X : x_data, Y : y_data}))

10 0.83956414
20 0.60977596
30 0.4477973
40 0.33564448
50 0.25590932
60 0.1955425
70 0.14761533
80 0.11136001
90 0.08366016
100 0.06298254
예측값 : [0 1 2 0 0 2]
실제값 : [0 1 2 0 0 2]
정확도 : 100.0
